# FooTrickster
author : Mer Win, Cheong

This project is attempting to "get/unlock" the Google's FooBar Challenge (hopefully reliably) by "tricking" the google's algorithm into thinking the user is a Python Pro. (Provided that the foobar challenge is still around(?))

### Background
The Google's FooBar Challenge is a secret invitation-only Pythonic/Java-nic(?) challenge that is issued to a few(?) selected user that exhibit proficiency in the Python/Java programming language. User who have been selected are typically shown the following screen (typically after their search results)

![GoogleFooBarInvitation]("FooExample.Jpeg")

### Methodology
The following notebook attempts to "trick" google's algorithm by executing the following:
1. Randomly ask "Pythonic" questions (provided by Stack Overflow and it's user, all credits go to them under the Creative Commons License, questions source can be found at the Kaggle webpage : https://www.kaggle.com/stackoverflow/pythonquestions#Questions.csv).
2. Capture screenshot of search page.
3. Use autoencoder model to predict if FooBar Challenge has been extended to us.
4. If invited, go to step **7**. Else, continue step **5**
5. Randomly click 0/1/2 link(s) in search results. Randomly wait 5/10/15 seconds after click).
6. Repeat step **1**
7. Analyze log files for types/frequency of questions asked (perhaps with clustering algorithm / NLP tokenization)

In [28]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
import datetime
import cv2

In [2]:
questionDF = pd.read_csv(r"./Data/Questions.csv", encoding="ISO-8859-1")
questionDF.head(5)

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [3]:
# Taking the smarter questions
smartscore = 40
QuestionList = list(questionDF[questionDF["Score"] > smartscore]["Title"])
print('There are {0} questions that has higher than {1} score'.format(len(QuestionList),smartscore))

There are 3593 questions that has higher than 40 score


In [4]:
# Generate random string to build a non-python question database
randomquestion = [i+j+k for i in 'ABCDEFGHIJK' for j in 'ABCDEFGHIJK' for k in 'ABCDEFGHIJK']
len(randomquestion)

1331

In [41]:
# Collecting images of random questions searches to build an extremely-biased outlier encoder model
for i in np.arange(len(randomquestion)):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("user-data-dir=C:/Users/Innovations/AppData/Local/Google/Chrome/User Data")
    browser = webdriver.Chrome(r"C:\Users\Innovations\Downloads\chromedriver_win32\chromedriver", chrome_options=options)
    browser.get("http://www.google.com")
    search_box = browser.find_element_by_name('q')
    search_box.send_keys(randomquestion[i])
    search_box.submit()
    time.sleep(2)
    browser.save_screenshot("./Data/NotFoo/{0:04}.png".format(i))
    browser.quit()

C:\Users\Innovations\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [6]:
import tensorflow as tf
def decode_image(filename, image_size=(56, 56)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_png(bits, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.crop_to_bounding_box(image, 50,150,500,1400)
    image = tf.image.resize(image, image_size)
    image = tf.expand_dims(image, axis=0)
    
    return image

In [7]:
from tensorflow.keras.models import load_model
fooModel = load_model("FooModel.h5")

In [8]:
fooModel.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 56, 56, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 32)                420640    
_________________________________________________________________
decoder (Model)              (None, 56, 56, 1)         470017    
Total params: 890,657
Trainable params: 890,273
Non-trainable params: 384
_________________________________________________________________


In [24]:
logfile = "./log.txt"
PATH = "C:/Users/Innovations/Desktop/AI/Trickception/"
with open(logfile, mode="w") as f:
    f.write("Datetime,Index,Question,MSE\n")
    f.close()

In [49]:
waitingTime = [5, 10, 15, 20]
with open(logfile, mode="a+", encoding='iso-8859-1') as f:
    for i in np.arange(363,len(QuestionList)):
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument("user-data-dir=C:/Users/Innovations/AppData/Local/Google/Chrome/User Data")
        browser = webdriver.Chrome(r"C:\Users\Innovations\Downloads\chromedriver_win32\chromedriver", chrome_options=options)
        
        browser.get("http://www.google.com")
        search_box = browser.find_element_by_name('q')
        question = np.random.choice(QuestionList)
        search_box.send_keys(question)
        search_box.submit()
        
        browser.save_screenshot("./Data/Foo/python_{0:04}.png".format(i))
        image = decode_image(PATH + "Data/Foo/python_{0:04}.png".format(i))
        decoded = fooModel.predict(image)
        mse = np.mean((image - decoded) ** 2)
        thres = mse > 0.1
        f.write(datetime.datetime.now().strftime("%c") + "," + str(i) + "," + question + "," + str(mse) + "\n")
        f.flush()
        
        if thres:
            input("FooChallenge Unlocked")
            browser.quit()
        else:
            if abs(np.random.randn(1)) > 0.5:
                results = browser.find_elements_by_xpath('//div[@class="r"]/a/h3')
                results[0].click()
                time.sleep(np.random.choice(waitingTime))
                browser.back()
                if abs(np.random.randn(1)) > 0.8:
                    myElem = WebDriverWait(browser, 3)
                    results = browser.find_elements_by_xpath('//div[@class="r"]/a/h3')
                    try:
                        results[1].click()
                        time.sleep(np.random.choice(waitingTime))
                    except:
                        pass
            time.sleep(1)
            browser.quit()

C:\Users\Innovations\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


FooChallenge Unlocked 
FooChallenge Unlocked 
FooChallenge Unlocked
FooChallenge Unlocked
FooChallenge Unlocked
FooChallenge Unlocked
FooChallenge Unlocked
FooChallenge Unlocked
FooChallenge Unlocked


KeyboardInterrupt: 